<a href="https://colab.research.google.com/github/vocaltrainer/pipeline_woo/blob/main/woo_pipeline(%EB%B9%84%EC%A0%95%ED%98%95).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 경로

In [ ]:
data_path = '/kaggle/input/chest-xray-pneumonia/chest_xray/'
train_path = data_path+'train/'
valid_path = data_path+'val/'
test_path = data_path+'test/'

# 데이터 읽기

In [ ]:
train = pd.read_csv(train_path)
test = pd.read_csv(valid_path)
test = pd.read_csv(test_path)

#EDA(비정형 data)

## 데이터 수

디렉터리가 여러개 있는 경우

In [ ]:
from glob import glob
print(f'train data 수: {len(glob(train_path+"*/*"))}')
print(f'valid data 수: {len(glob(valid_path+"*/*"))}')
print(f'test data 수: {len(glob(test_path+"*/*"))}')

한 csv파일인 경우

In [ ]:
train.shape, test.shape

## 데이터 라벨 분포

In [ ]:
path_list = [train_path,valid_path,test_path]
normal = []
pneu = []
for path in path_list:
    normal.extend(glob(path+'NORMAL/*'))
    pneu.extend(glob(path+'PNEUMONIA/*'))
print(f'normal 수: {len(normal)}')
print(f'pneumonia 수: {len(pneu)}')

In [ ]:
import matplotlib.pyplot as plt

label = ['normal','pneumonia']
plt.pie([len(normal),len(pneu)],labels=label,autopct='%.1f')

## 이미지 이름으로부터 이미지 읽기

In [ ]:
import cv2

def read_image(img_id):
        img_path = f'{data_path}/images/{img_id}.jpg'
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        return image

## 열(이미지 이름) -> 열(이미지) 로 변환

In [ ]:
multi_id = multi['image_id'].reset_index()
multi_img = multi_id['image_id'].apply(read_image)

## 열(이미지)의 RGB 채널 분리

In [ ]:
import numpy as np

red_values = [np.mean(multi_img[idx][:,:,0]) for idx in range(len(multi_id))]
green_values = [np.mean(multi_img[idx][:,:,1]) for idx in range(len(multi_id))]
blue_values = [np.mean(multi_img[idx][:,:,2]) for idx in range(len(multi_id))]
values = [np.mean(multi_img[idx]) for idx in range(len(multi_id))]

## 전체 데이터의 RGB 채널별 픽셀 분포

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

fig, ax = plt.subplots(2,2,figsize=(15,15))
sns.distplot(values,ax=ax[0,0])
sns.distplot(red_values, ax=ax[0,1])
sns.distplot(green_values, ax=ax[1,0])
sns.distplot(blue_values, ax=ax[1,1])

ax[0,0].set_title(" multi_values(rgb)")
ax[0,1].set_title("red")
ax[1,0].set_title("green")
ax[1,1].set_title("blue")

## 이미지에서 객체 탐지(Canny)및 출력

탐지

In [ ]:
def edge_and_cut(img):
    emb_img = img.copy()
    edges = cv2.Canny(img, 250, 300)
    edge_coors = []
    for i in range(edges.shape[0]):
        for j in range(edges.shape[1]):
            if edges[i][j] != 0:
                edge_coors.append((i, j))

    row_min = edge_coors[np.argsort([coor[0] for coor in edge_coors])[0]][0]
    row_max = edge_coors[np.argsort([coor[0] for coor in edge_coors])[-1]][0]
    col_min = edge_coors[np.argsort([coor[1] for coor in edge_coors])[0]][1]
    col_max = edge_coors[np.argsort([coor[1] for coor in edge_coors])[-1]][1]
    new_img = img[row_min:row_max, col_min:col_max]

    emb_img[row_min-10:row_min+10, col_min:col_max] = [255, 0, 0]
    emb_img[row_max-10:row_max+10, col_min:col_max] = [255, 0, 0]
    emb_img[row_min:row_max, col_min-10:col_min+10] = [255, 0, 0]
    emb_img[row_min:row_max, col_max-10:col_max+10] = [255, 0, 0]

    return emb_img

출력

In [ ]:
def show_image_bb(img_ids, rows=2, cols=3):
    #assert len(img_ids) <=rows*cols
    f, ax = plt.subplots(rows, cols,figsize=(15,8))

    for idx,img_id in enumerate(img_ids):
        img_path = f'{data_path}/images/{img_id}.jpg'
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        ax[0,idx].imshow(image)
        ax[1,idx].imshow(edge_and_cut(image))
    plt.show()

In [ ]:
last = multi_id['image_id'][-3:].tolist()
show_image_bb(last)

##경로로부터 이미지 출력(6개 이미지)

In [ ]:
import cv2

def show_image(img_ids, rows=2, cols=3):
    #assert len(img_ids) <=rows*cols
    f, ax = plt.subplots(rows, cols,figsize=(15,8))

    for idx,img_id in enumerate(img_ids):
        img_path = img_ids[idx]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        ax[idx//3,idx%3].imshow(image)
    plt.show()

In [ ]:
last_normal = list(normal[-6:])
show_image(last_normal)

# 시드값 고정(재현성) 및 세팅

In [ ]:
import torch
import random
import numpy as np
import os

seed=50
os.environ['PYTHONHASHSEED']=str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic=True
torch.backends.cudnn.benchmark=False
torch.backends.cudnn.enabled=False

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

# train_test_split(stratify)

stratify로 타깃값이 골고루 분포되도록 분리

In [ ]:
from sklearn.model_selection import train_test_split

#stratify로 타깃값이 골고루 분포되도록 분리
train,valid = train_test_split(train,test_size=0.1,
                               stratify=train[['healthy','multiple_diseases',
                                               rust','scab']],
                               random_state=50)

# data augmentation, dataset 생성

## 디렉터리로 라벨링이 되있는 경우

In [ ]:
from torchvision import transforms

transform_train = transforms.Compose([transforms.Resize((250,250)),
                                     transforms.CenterCrop(180),
                                     transforms.RandomHorizontalFlip(0.5),
                                     transforms.RandomVerticalFlip(0.2),
                                     trnasforms.RandomRotation(20),
                                     transforms.ToTensor(),
                                     transforms.Normalize((0.485,0.456,0.406),
                                                         (0.229,0.224,0.255))])   #imageNet이 학습한 각 채널의 평균과 표준편차. 이를 사용하는것이 성능에 도움이됨

transform_test = transforms.Compose([transforms.Resize(250,250),
                                     transforms.CenterClip(180),
                                     transforms.ToTensor(),

                                     transforms.Normalize((0.485,0.456,0.406),
                                                         (0.229,0.224,0.255))])

SyntaxError: ignored

In [ ]:
from torchvision.datasets import ImageFolder
#albumentation 쓰면 오류. torchvision.transforms 써야 함.
#라벨링값이 같은 데이터들이 같은 디렉터리에 모여있어야 ImageFolder 쓸 수 있음.

datasets_train = ImageFolder(root=train_path, transform = transform_train)
datasets_test = ImageFolder(root=test_path,transform = transform_test)

## 그렇지 않은 경우 사용자 정의 dataset 생성

albumentations : torchvision.transforms보다 많은 기능 지원

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

In [ ]:
transform_train = A.Compose([
    A.Resize(450,650),                                       #이미지를 크게 조정하면 성능이 좋다고 함. 단, 그만큼 리소스가 커짐
    A.RandomBrightnessContrast(brightness_limit=0.2,             #brightness:밝기, contrast:대비, p:확률
                          contrast_limit=0.2,p=0.3),
    A.VerticalFlip(p=0.2),                                   #상하 대칭 변환
    A.HorizontalFlip(p=0.5),                                 #좌우 대칭 변환
    A.ShiftScaleRotate(                                      #이동, 스케일링, 회전각도 조절
    shift_limit=0.1,
    scale_limit=0.2,
    rotate_limit=30,p=0.3),
    A.OneOf([A.Emboss(p=1),                                  #양각화, 날카롭게 만드는 효과, 블러 효과 중 한가지를 적용
            A.Sharpen(p=1),
            A.Blur(p=1)],p=0.3),
    A.PiecewiseAffine(p=0.3),                                #어파인 변환기(이동,확대/축소, 회전등으로 이미좀양을 전체적으로 바꿈)
    A.Normalize(),                                           #정규화
    ToTensorV2()                                             #이미지데이터를 텐서로 변환

])

transform_test = A.Compose([A.Resize(450,650),    #Train data와 크기를 같게
                           A.Normalize(),         #픽셀값 범위도 같게
                           ToTensorV2()])

NameError: ignored

In [ ]:
import cv2
from torch.utils.data import Dataset
import numpy as np

class ImageDataset(Dataset):
    def __init__(self, df,img_dir='/',transform=None, is_test=False):
        super().__init__()
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
        self.is_test = is_test
    def __len__(self):
        return len(self.df)

    def __getitem__(self,idx):
        img_id = self.df.iloc[idx,0]
        img_path = self.img_dir+img_id+'.jpg'
        img = cv2.imread(img_path)
        image = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

        if self.transform is not None:
            image =self.transform(image=image)['image']  #albumentation 사용할 것임

        if self.is_test:
            return image
        else:
            label = np.argmax(self.df.iloc[idx,1:5])
            return image, label

In [ ]:
img_dir = f'{data_path}/images/'
dataset_train = ImageDataset(train,img_dir=img_dir,transform=transform_train)
dataset_valid = ImageDataset(valid,img_dir=img_dir,transform=transform_test)
dataset_test = ImageDataset(test, img_dir = img_dir,
                           transform = transform_test, is_test=True)

#dataloader 시드고정(멀티프로세싱을 위해)

In [ ]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed()%2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(0)

# dataloader 생성

In [ ]:
from torch.utils.data import DataLoader

batch_size = 4
loader_train = DataLoader(datasets_train, batch_size = batch_size, shuffle=True, worker_init_fn = seed_worker,
                         generator = g, num_workers=2)

loader_valid = DataLoader(datasets_valid, batch_size = batch_size, shuffle=False, worker_init_fn = seed_worker,
                          generator = g, num_workers = 2)
loader_test = DataLoader(dataset_test, batch_size=batch_size,
                           shuffle=False, worker_init_fn=seed_worker,generator=g,num_workers=2)

# EfficientNet 모델 생성

In [ ]:
!pip install efficientnet-pytorch==0.7.1

In [ ]:
from efficientnet_pytorch import EfficientNet

model = EfficientNet.from_pretrained('efficientnet-b7',num_classes=4)  #num_classes: 현 competition의 출력값개수는 4개!
model = model.to(device) #GPU 할당

참고로 아래와 같은 방법으로 EfficientNet의 마지막 FC는 수정이 가능하다.

In [ ]:
# import torch.nn as nn

# model = EfficientNet.from_pretrained('efficientnet-b7')
# model._fc=nn.Sequential(nn.Linear(model._fc.in_features,model._fc.out_features),
#                         nn.ReLU(),
#                         nn.Dropout(0.5),
#                         nn.Linear(Model._fc.out_features,4)
#                        )

# loss, optimizer 설정

In [ ]:
import torch.nn as nn
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(),lr=0.00006,weight_decay=0.0001)

# Train

In [ ]:
from sklearn.metrics import roc_auc_score
from tqdm.notebook import tqdm
epochs=5

for epoch in range(epochs):
    model.train()              #train 상태로 설정
    epoch_train_loss = 0

    for images, labels in tqdm(loader_train):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        epoch_train_loss += loss.item()
        loss.backward()
        optimizer.step()
    print(f'epoch[{epoch+1}/{epochs}] -train 데이터 손실값 : {epoch_train_loss/len(loader_train):.4f}')

    #미니배치 단위로 검증 수행
    model.eval()
    epoch_valid_loss=0
    preds_list=[]
    true_onehot_list=[]

    with torch.no_grad():
        for images, labels in loader_valid:
            images = images.to(device)
            labels = labels.to(device)

            outputs=model(images)
            loss = criterion(outputs, labels)
            epoch_valid_loss += loss.item()

            preds = torch.softmax(outputs.cpu(),dim=1).numpy()
             #원핫인코딩, device='cuda'해줘야 함
            true_onehot = torch.eye(4,device='cuda')[labels].cpu().numpy()

            preds_list.extend(preds)
            true_onehot_list.extend(true_onehot)

        print(f'epoch[{epoch+1}/{epochs}] -valid 데이터 손실값 : {epoch_valid_loss/len(loader_valid):.4f} / valid ROC_AUC : {roc_auc_score(true_onehot_list,preds_list):.4f}')

# Test

In [ ]:
model.eval()

preds = np.zeros((len(test),4))

with torch.no_grad():
    for i,images in enumerate(loader_test):
        images = images.to(device)
        outputs = model(images)
        preds_part = torch.softmax(outputs.cpu(),dim=1).squeeze().numpy()
        preds[i*batch_size:(i+1)*batch_size]+=preds_part

#기타. 캐글 submission

In [ ]:
data_path = '/kaggle/input/plant-pathology-2020-fgvc7/'
submission = pd.read_csv(data_path+'sample_submission.csv')

In [ ]:
submission[['healthy','multiple_diseases','rust','scab']]=preds

In [ ]:
submission.to_csv('submission1.csv',index=False)